In [ ]:
# import libraries
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# hyperparameters
batch_size = 32  # bir seferde paralel işlenecek bağımsız dizi sayısı
block_size = 8   # tahminler için maksimum bağlam uzunluğu
max_iters= 3000   # eğitim için maximum iterasyon sayısı
eval_interval = 300  # Değerlendirme aralığı
learning_rate = 1e-2  # Öğrenme oranı
device = 'cuda' if torch.cuda.is_available() else 'cpu' # cihaz gpu varsa cuda yoksa cpu da çalışacak
eval_iters = 200 # değerlendirme için iterasyon sayısı

- batch_size: Eğitim sırasında aynı anda işlenecek örnek sayısı. 32, her iterasyonda 32 farklı metin parçasının paralel olarak işleneceği anlamına gelir. Bu, eğitimi hızlandırır ve modelin genelleme yapmasına yardımcı olur.
- block_size: Modelin bir seferde görebileceği maksimum metin uzunluğu (karakter cinsinden). Burada 8 karakterlik bir bağlam kullanılıyor. Örneğin, model "Merhaba" kelimesini tahmin ederken en fazla 8 karakter geriye bakabilir.
- max_iters: Modelin kaç kez eğitileceği (toplam iterasyon sayısı). 3000 iterasyon boyunca model, veriyi tekrar tekrar işleyecek.
- eval_interval: Modelin performansını değerlendirmek için kaç iterasyonda bir kontrol yapılacağı. Her 300 iterasyonda bir, modelin doğruluğu kontrol edilir.
- learning_rate: Modelin ağırlıklarını ne kadar hızlı güncelleyeceği. 0.01 (1e-2), oldukça standart bir öğrenme oranıdır.
- device: Eğitimde kullanılacak donanım. Eğer GPU (CUDA) varsa, eğitim hızlanır; yoksa CPU kullanılır.
- eval_iters: Değerlendirme sırasında kaç iterasyon yapılacağı. 200 iterasyon, modelin doğruluğunu test etmek için yeterli bir örnek sağlar.

In [ ]:
torch.manual_seed(1337) # rastgelelik kontrolü. Model her çalıştırıldığın aynı rastgele sayıları üretir

In [ ]:
# Veri setini indirme ve okuma
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt','r',encoding = 'utf-8') as f:
  text = f.read()

--2025-06-12 09:48:58--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-06-12 09:48:58 (17.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
print('Length of dataset in characters',len(text))

Length of dataset in characters 1115394


In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# Metindeki tüm benzersiz karakterler

chars = sorted(list(set(text)))
vocab_size = len(chars)

# Karakterden sayıya ve sayıdan karaktere eşleştirme
stoi = {ch : i for i,ch in enumerate(chars)}
itos = {i : ch for i , ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s]   # encoder : take a string , output a list of integer ---- Stringi al karakter listesindeki her stringe karşılık gelen sayılara dönüştür
decode = lambda l : ''.join([itos[i] for i in l]) # decoder: take a list of integers , output a string ---> İntegerların oluşturduğu listeleri al ve her integera karşılık gelen harfe dönüştür
print('Benzersiz karakter sayısı : ',vocab_size)
print(stoi)
print(itos)

Benzersiz karakter sayısı :  65
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44:

- Benzersiz karakterler: Metindeki tüm farklı karakterler (a, b, ,, ., boşluk, vb.) bulunur ve sıralanır. set(text) ile tekrar eden karakterler kaldırılır, sorted() ile alfabetik sıraya dizilir
- vocab_size: Metindeki benzersiz karakter sayısı (örneğin, 65 olabilir, çünkü İngilizce metinlerde genellikle harfler, noktalama işaretleri ve birkaç özel karakter bulunur)

- stoi (String to Integer): Her karaktere bir tamsayı atar. Örneğin, 'a' → 0, 'b' → 1, ' ' → 2 gibi. Bu, metni sayısal forma çevirmek için kullanılır.
- itos (Integer to String): Tamsayıları tekrar karaktere çevirir. Örneğin, 0 → 'a', 1 → 'b'.
- encode: Bir metni (string) alır ve her karakteri stoi kullanarak bir tamsayı listesine çevirir. Örneğin, "hi" → [1, 2]).
- decode: Bir tamsayı listesini decode kullanarak tekrar metne çevirir (örneğin, [1, 2] → "hi"`).

In [ ]:
# train and test split
data = torch.tensor(encode(text) , dtype = torch.long) # Metin verisini sayısal forma dönüştürme
n = int(0.9*len(data))  # datasetin %90 eğitim için kullanılacak
train_data = data[:n]  # Metnin %90'ı eğitim verisi
val_data = data[n:]   #Metnin %10 'u doğrulama verisi

In [ ]:
# Veri yığını (batch) oluşturmak için fonksiyon
def get_batch(split):
    # Hangi veri setini kullanacağımızı seçiyoruz: eğitim('train) veya doğrulama ('val)
    data = train_data if split == 'train' else val_data
    # Rastgele başlangıç indeksleri seçiyoruz(batch size kadar , veri uzunluğundan block_size çıkararak)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # Giriş verisi (x) : Her bir başlangıç indeksinden block_size kadar karakter alıyoruz
    x = torch.stack([data[i:i+block_size] for i in ix])
    # Hedef verisi (y) : x 'in bir karakter kaydırılmış hali (sonraki karakteri tahmin edeceğiz)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # Verileri seçilen cihaza (GPU veya CPU ya) yaşıyoruz
    x, y = x.to(device), y.to(device)
    # Giriş (x) ve hedef (y) tensorlarını döndürüyoruz
    return x, y

- Amaç: Eğitim veya doğrulama için rastgele bir veri yığını (batch) üretir. Her bir batch, batch_size kadar örnek içerir ve her örnek block_size uzunluğundadır.
- split: Hangi veri setinin kullanılacağını belirtir ('train' veya 'val').
- ix: Veri setinden rastgele başlangıç indeksleri seçer. len(data) - block_size ile veri sonuna taşmamak için sınırlandırılır.
- x: Giriş verisi, her biri block_size uzunluğunda tamsayı dizileridir (örneğin, "hello"nun kodlanmış hali: [0, 1, 2, 2, 3]).
- y: Hedef verisi, xin bir karakter kaydırılmış halidir. Örneğin, x = [0, 1, 2, 2, 3] ise, y = [1, 2, 2, 3, ?] olur (model bir sonraki karakteri tahmin eder).
- to(device): Veriler, GPU (cuda) veya CPU'ya taşınır, böylece modelin çalışacağı donanıma uygun olur.
- Dönen değer: x (giriş) ve y (hedef) tensorları, şekilleri (batch_size, block_size).

In [ ]:
# Modelin kaybını (loss) değerlendirmek için fonksiyon (gradyan hesaplaması olmadan)
@torch.no_grad()  # gradyan izlemeyi durdur

def estimate_loss(): # Tahmini kayıp
  #Sonuçları saklamak için bir sözlük
  out = {}
  # modeli değerlendirme moduna geçir
  model.eval()
  # Eğitim ve doğrulama setleri için bir döngü başlatılır
  for split in ['train','val']:
    # eval_iters(200) kadar kayıp değerini saklamak için sıfırlardan oluşan bşr tensor
    losses = torch.zeros(eval_iters)

    # eval iters kadar döngü
    for k in range(eval_iters):
      # Rastgele bir veri yığını al
      X , Y = get_batch(split)
      # modeli çalıştırarak logits ve kaybı hesapla
      logits , loss = model(X,Y)
      # kaybı kaydet
      losses[k] = loss.item()
    # Ortalama kaybı sözlüğe ekle
    out[split] = losses.mean()
  # modeli tekrar eğitim moduna geçir
  model.train()
  # eğitim ve doğrulama kayıplarını döndür
  return out



- @torch.no_grad(): Gradyan hesaplamasını devre dışı bırakır, böylece değerlendirme sırasında bellek ve hesaplama tasarrufu yapılır.
- model.eval(): Modeli değerlendirme moduna alır (örneğin, dropout veya batch normalization gibi katmanlar farklı davranır)
- split döngüsü: Hem eğitim (train) hem de doğrulama (val) setleri için kayıp hesaplanır
- losses: eval_iters kadar kayıp değerini saklar. Her iterasyonda yeni bir batch alınır ve modelin kaybı hesaplanır
- logits, loss: Model, giriş (X) ve hedef (Y) verileriyle çalıştırılır. logits, modelin ham tahminlerini; loss, tahminlerin ne kadar yanlış olduğunu gösterir.
- losses.mean(): eval_iters boyunca hesaplanan kayıpların ortalaması alınır.
- model.train(): Model, değerlendirme bittikten sonra tekrar eğitim moduna geçirilir.
- Dönen değer: Eğitim ve doğrulama setleri için ortalama kayıpları içeren bir sözlük ({'train': ..., 'val': ...})

In [ ]:
# Çok basit bir bigram dil modeli sınıfı

class BigramLanguageModel(nn.Module):
  #Modelin başlatılması
  def __init__(self,vocab_size):
    super().__init__()
    # Her token , bir sonraki token için logits'i bir tablodan doğrudan okur
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  #Modelin ileri geçiş(forward pass) fonksiyonu
  def forward(self,idx , targets = None):
    # idx ve targets , tam sayılardan oluşan (B , T) boyutlu tensorlardır

    logits = self.token_embedding_table(idx) # (B , T , C) boyutlu tensor haline gelir
    # Eğer hedef verimezse kayıp hesaplanmas
    if targets is None:
      loss = None

    else:
      # logits ve targers tensorlerini uygun şekle getir
      B , T , C = logits.shape
      logits = logits.view(B*T , C) # (B*T,C) şekline düzleştir --- Uzatıyoruz yukarıdan aşağıya
      targets = targets.view(B*T)  # (B*T) şekline düzleştir
      # Çapraz entropi kaybını hesapla
      loss = F.cross_entropy(logits , targets)
    # Logits ve kaybı döndür
    return logits , loss

  #metin üretmek için fonksiyon
  def generate(self , idx,max_new_tokens):
    #idx : mevcut bağlamdaki indekslerin (B , T)  boyutlu dizisi
    # max_new_tokens : üretilecek maksimum yeni token sayısı
    for _ in range(max_new_tokens): # Belirtilen sayıda yeni token üret

      # tahminleri al
      logits, loss = self(idx)
      # sadece son zaman adımına odaklan
      logits = logits[: , -1 ,:] # (B,C ) şekline getirir(son token'ın logitsleri)
      # Olasılıklara dönüştürmek için softmax uygula
      probs = F.softmax(logits , dim = -1)  # (B , C) , her token için olasılık dağılımı
      # Olasılık dağılımından rastgele bir token seç
      idx_next = torch.multinomial(probs , num_samples = 1) # (B,1) bir sonraki token
      # Seçilen tokeni mevcut diziye ekle
      idx = torch.cat((idx , idx_next) , dim = 1)  # (B , T + 1) , diziyi güncelle
    # Üretilen token dizisini döndür
    return idx



- Sınıf tanımı: BigramLanguageModel, PyTorch'un nn.Module sınıfından türetilir. Bu, bir sinir ağı modelidir.
- init:
  - vocab_size: Metindeki benzersiz karakter sayısı (önceki kodda hesaplandı).
  - nn.Embedding(vocab_size, vocab_size): Her karakter için bir gömme (embedding) vektörü oluşturur. Bu, bir tablo gibi çalışır: her karakter (token), bir sonraki karakterin olasılık dağılımını temsil eden bir vektöre (logits) eşlenir.
  - Örneğin, eğer vocab_size = 65, her karakter için 65 boyutlu bir vektör (her biri bir sonraki karakterin olasılığını temsil eder) oluşturulur.
- forward:
  - idx: Giriş tensoru, şekli (B, T) (batch_size, block_size). Her eleman, bir karakterin tamsayı kodlamasıdır.
  - targets: Hedef tensoru, şekli (B, T). Bir sonraki karakterlerin tamsayı kodlamalarını içerir.
  - logits: nn.Embedding tablosundan her giriş token’ı için bir vektör alınır. Çıktı şekli: (B, T, C) (batch_size, block_size, vocab_size).
- Kayıp hesaplama:
  - Eğer targets yoksa, sadece logits döndürülür (örneğin, tahmin yaparken).
  - Eğer targets varsa:
    - logits ve targets düzleştirilir (view ile) çünkü cross_entropy fonksiyonu 2D giriş bekler.
    - logits: (B*T, C) (her token için olasılık vektörü).
    - targets: (B*T) (her token için doğru sonraki karakter).
    - F.cross_entropy: Modelin tahminleri (logits) ile gerçek değerler (targets) arasındaki kaybı hesaplar.
    - Dönen değer: logits, loss (tahminler ve kayıp).

- self : BigramLanguageModel sınıfının bir örneği(model)
- idx : Mevcut bağlam , şekli (B ,T ) ---> (batch_size , mevcut bağlam uzunluğu).Her eleman , bir karakterin tam sayı kodlamasıdır.
- max_new_token : Kaç yeni karakter üretileceğini belirtir.
- logits , loss : Model , bağlam(idx) ile çalıştırılır ve her token için bir sonraki tokenın olasılıkları(logits)döndürür.Kayıp burada kullanılmaz.
- logits[: , -1 , :] : Sadece son zaman adımının logitsleri alınır.(Çünkü bigram modelinde bir sonraki tokeni tahmin ediyoruz).Şekil (B , C ) olur.---> (C : vocab_size)
- F.softmax: Logits'leri olasılıklara çevirir. dim=-1, son boyutta (vocab_size) softmax uygular.
- torch.multinomial: Olasılık dağılımından rastgele bir token seçer. Örneğin, 'a' için yüksek olasılık varsa, onu seçme ihtimali daha yüksektir.
- torch.cat: Yeni seçilen token (idx_next) mevcut bağlama eklenir, böylece bağlam büyür.
- Dönen değer: Üretilen token dizisi, şekli (B, T+max_new_tokens).

In [ ]:
# Bigram sil modelini oluştur
model = BigramLanguageModel(vocab_size)
# Modeli seçilen cihaza (GPU veya CPU) taşı
m = model.to(device)
#Pytorch optimizasyon aracını oluştur
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

- model: BigramLanguageModel sınıfından bir model örneği oluşturulur. vocab_size, metindeki benzersiz karakter sayısına dayanır (önceki kodda hesaplandı).
- to(device): Model, GPU (cuda) veya CPU'ya taşınır, böylece eğitim donanıma uygun olur.
- optimizer: AdamW optimizatörü, modelin parametrelerini (örn. token_embedding_table) güncellemek için kullanılır. learning_rate, hiperparametrelerden gelir (1e-2 = 0.01).
- model.parameters(): Modelin öğrenilebilir parametrelerini (gömme tablosu) optimizatöre verir.

In [ ]:
# Eğitim için belirtilen iterasyon kadar döngü
for iter in range(max_iters):

    # Her eval_iterval iterasyonda kaybı değerlendir
    if iter % eval_interval == 0:
        losses = estimate_loss()  # eğitim ve doğrulama kayıplarını hesapa
        # adım numarası ve kayıpları yazdır( 4 ondalık basamakla)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Eğitim verisinden bir veri yığını al
    xb, yb = get_batch('train')

    # Kaybı hesapla
    logits, loss = model(xb, yb) #modeli çalıştırarak tahmin ve kayıp al
    optimizer.zero_grad(set_to_none=True) #Önceki gradyanları sıfırla
    loss.backward() # Gradyanları hesapla
    optimizer.step() # Model parametrelerini güncelle

step 0: train loss 4.5847, val loss 4.5811
step 300: train loss 2.7966, val loss 2.8134
step 600: train loss 2.5460, val loss 2.5566
step 900: train loss 2.5060, val loss 2.5202
step 1200: train loss 2.4792, val loss 2.5025
step 1500: train loss 2.4744, val loss 2.4939
step 1800: train loss 2.4703, val loss 2.4842
step 2100: train loss 2.4635, val loss 2.4940
step 2400: train loss 2.4577, val loss 2.4914
step 2700: train loss 2.4707, val loss 2.4876


- max_iters: Eğitim için toplam iterasyon sayısı (örneğin, 3000).
- eval_interval: Her 300 iterasyonda bir, modelin performansı (estimate_loss ile) kontrol edilir ve eğitim/doğrulama kayıpları yazdırılır.
get_batch('train'): Eğitim verisinden rastgele bir batch alınır (xb: giriş, yb: hedef).
log    its, loss: Model, giriş (xb) ile çalıştırılır ve tahminler (logits) ile kayıp (loss) hesaplanır.
- optimizer.zero_grad(): Önceki gradyanlar sıfırlanır, böylece yeni gradyanlar temiz bir şekilde hesaplanır.
- loss.backward(): Kayıp üzerinden gradyanlar hesaplanır (geri yayılım).
- optimizer.step(): Gradyanlar kullanılarak model parametreleri güncellenir (örneğin, gömme tablosu).
- print: Her eval_interval adımda, eğitim ve doğrulama kayıpları yazdırılır, böylece modelin öğrenme süreci izlenir.

In [ ]:
# modelden metin üret
context = torch.zeros((1,1), dtype = torch.long , device = device) # Başlangıç bağlamı  : boş bir token (sıfır)
# 500 yeni tken üret ve kodlanmış tokenları metne çevir
print(decode(m.generate(context , max_new_tokens=500)[0].tolist()))


Whey nesee miesosent alls ols s. t he pe hqu neerete

Ifo p bleyo laun mpest?
KEYOProrant mepr ind theak twieakes t fusco then f rere ben bem h an INClelimare!

FassordBunthin'stise ittoof nous nd t.
Nof lerin.

ourllaclors, the ou d tharellele glyorm
Thikik se br,
LQUThewouithod Gop
BRicinth?
ETouize wrattharoungie lastngoeyon a K:
KENUCo nk malarse. de augenesard d, pilldy as, de macoorlf r st.
O:
G dindfoe pas t ardir o t war Cof jofucouo ERDiat twin f atul, I'rmendiouthangericotharkeen
CLARI


- context: Başlangıç bağlamı, (1, 1) boyutlu bir tensor (tek batch, tek token). torch.zeros ile sıfırla başlatılır (örneğin, metnin başındaki "boş" bir karakter).
- m.generate: Model, context ile başlayarak 500 yeni token üretir. generate fonksiyonu, her adımda bir sonraki tokenı tahmin eder ve bağlama ekler.
[0].tolist(): Üretilen token dizisi, ilk batch’ten alınır ([0]) ve Python listesine çevrilir.
- decode: Tamsayı token’ları ([0, 1, 2, ...]) metne çevrilir (örneğin, "hello"). Bu, önceki kodda tanımlı itos (integer to string) ile yapılır.
- print: Üretilen metin ekrana yazdırılır.

**Bigram Modelinin Sınırlamaları:**

- Bigram modeli, yalnızca bir önceki karaktere bakar, bu yüzden uzun vadeli bağlamları yakalayamaz (örneğin, cümle yapıları veya hikaye tutarlılığı).
- Üretilen metin, genellikle rastgele ve anlamsız olabilir, çünkü model çok basittir.